In [1]:
import os
# import face_recognition_api
import face_recognition as face_recognition_api
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np
import pandas as pd

In [2]:
def _get_training_dirs(training_dir_path):
    return [x[0] for x in os.walk(training_dir_path)][1:]


def _get_training_labels(training_dir_path):
    return [x[1] for x in os.walk(training_dir_path)][0]


def _get_each_labels_files(training_dir_path):
    return [x[2] for x in os.walk(training_dir_path)][1:]


def _filter_image_files(training_dir_path):
    exts = [".jpg", ".jpeg", ".png"]

    training_folder_files_list = []
    for list_files in _get_each_labels_files(training_dir_path):
        l = []
        for file in list_files:
            imageName, ext = os.path.splitext(file)
            if ext.lower() in exts:
                l.append(file)
        training_folder_files_list.append(l)

    return training_folder_files_list


def _zipped_folders_labels_images(training_dir_path, labels):
    return list(zip(_get_training_dirs(training_dir_path),
                    labels,
                    _filter_image_files(training_dir_path)))


def create_dataset(training_dir_path, labels):
    X = []
    for i in _zipped_folders_labels_images(training_dir_path, labels):
        for fileName in i[2]:
            file_path = os.path.join(i[0], fileName)
            img = face_recognition_api.load_image_file(file_path)
#             X_faces_loc = face_recognition_api.face_locations(img, model="cnn")
            X_faces_loc = face_recognition_api.face_locations(img)
            imgEncoding = face_recognition_api.face_encodings(img, known_face_locations=X_faces_loc)

            if len(imgEncoding) > 1:
                print('\x1b[0;37;43m' + 'More than one face found in {}. Only considering the first face.'.format(file_path) + '\x1b[0m')
            if len(imgEncoding) == 0:
                print('\x1b[0;37;41m' + 'No face found in {}. Ignoring file.'.format(file_path) + '\x1b[0m')
            else:
                print('Encoded {} successfully.'.format(file_path))
                X.append(np.append(imgEncoding[0], i[1]))
    return X

In [3]:
encoding_file_path = './encoded-images-data.csv'
training_dir_path = './training-images'
labels_fName = "labels.pkl"

In [4]:
# Get the folder names in training-dir as labels
# Encode them in numerical form for machine learning
labels = _get_training_labels(training_dir_path)
le = LabelEncoder().fit(labels)
labelsNum = le.transform(labels)
nClasses = len(le.classes_)
dataset = create_dataset(training_dir_path, labelsNum)
df = pd.DataFrame(dataset)

Encoded ./training-images\胡瓜 Hu gua\005 (2).JPG successfully.
Encoded ./training-images\胡瓜 Hu gua\擷取.JPG successfully.
Encoded ./training-images\謝忻 Xie Xin\003.JPG successfully.
Encoded ./training-images\謝忻 Xie Xin\擷3取.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\00004.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\001.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\003.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\004.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\009.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\010.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\擷取9.JPG successfully.


In [5]:
df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,-0.027992,0.057353,0.026123,-0.060029,-0.030109,-0.055110,-0.037814,-0.112240,0.073034,-0.116441,...,0.173175,0.011552,0.057083,-0.237432,-0.031389,0.116258,0.021229,0.079483,-0.021711,0.0
1,-0.049811,0.055013,-0.012561,-0.029388,-0.061190,-0.036271,-0.011504,-0.103243,0.059203,-0.062347,...,0.144709,0.026924,-0.010054,-0.249223,-0.019650,0.165067,0.038910,0.097710,0.021790,0.0
2,-0.104630,0.071392,0.098343,-0.097491,-0.183588,-0.077647,-0.054400,-0.086117,0.111888,-0.155648,...,0.041416,0.000354,0.029422,-0.206591,-0.034639,0.110177,0.008536,0.044905,0.014981,1.0
3,-0.073589,0.051655,0.092356,-0.102369,-0.178207,-0.078088,-0.042658,-0.089103,0.116953,-0.175253,...,0.049721,-0.028193,0.044857,-0.210555,-0.029672,0.104281,0.003760,0.040472,0.000356,1.0
4,-0.122356,0.078419,0.023165,-0.047130,-0.172021,0.012583,-0.057570,-0.092241,0.125600,-0.056133,...,0.023869,-0.004250,-0.043355,-0.198941,-0.038010,0.087016,-0.055660,0.152364,0.032997,2.0
5,-0.081725,0.056925,0.036327,-0.036846,-0.124562,0.033040,-0.089305,-0.130996,0.103806,-0.135097,...,0.025850,0.007127,-0.039068,-0.232801,0.007201,0.122295,-0.014582,0.129896,-0.019757,2.0
6,-0.082705,0.069972,0.012787,-0.019849,-0.141806,-0.027747,-0.069015,-0.089459,0.117990,-0.077059,...,0.033567,-0.014620,-0.054208,-0.232799,-0.034721,0.089570,-0.010051,0.095630,-0.000803,2.0
7,-0.131376,0.090107,0.033875,-0.033487,-0.150795,0.028660,-0.091734,-0.121539,0.096089,-0.071615,...,0.049900,-0.009599,-0.023261,-0.234320,-0.022314,0.113547,-0.046847,0.112454,0.034447,2.0
8,-0.105776,0.053636,0.034845,-0.010332,-0.143507,-0.012405,0.001799,-0.135187,0.084832,-0.114182,...,-0.025877,0.001071,-0.046347,-0.263957,-0.065157,0.162250,-0.040819,0.175174,-0.018360,2.0
9,-0.083600,0.046998,0.048783,0.004225,-0.117413,-0.044493,-0.031472,-0.134362,0.080115,-0.103021,...,0.032563,0.005101,-0.047892,-0.251584,-0.067125,0.087115,-0.046976,0.104190,-0.018002,2.0


In [6]:
# if file with same name already exists, backup the old file
if os.path.isfile(encoding_file_path):
    print("{} already exists. Backing up.".format(encoding_file_path))
    try:
        os.rename(encoding_file_path, "{}.bak".format(encoding_file_path))
    except WindowsError:
        os.remove("{}.bak".format(encoding_file_path))
        os.rename(encoding_file_path, "{}.bak".format(encoding_file_path))
#    os.rename(encoding_file_path, "{}.bak".format(encoding_file_path))

df.to_csv(encoding_file_path)

print("{} classes created.".format(nClasses))
print('\x1b[6;30;42m' + "Saving labels pickle to'{}'".format(labels_fName) + '\x1b[0m')
with open(labels_fName, 'wb') as f:
    pickle.dump(le, f)
print('\x1b[6;30;42m' + "Training Image's encodings saved in {}".format(encoding_file_path) + '\x1b[0m')

./encoded-images-data.csv already exists. Backing up.
3 classes created.
Saving labels pickle to'labels.pkl'
Training Image's encodings saved in ./encoded-images-data.csv


In [7]:
import time
#####################
### create_encodings
stime = time.localtime()
print(time.strftime("%Y-%m-%d %H:%M:%S",stime))
for i in range(0,100000000,1):
    1+1
etime = time.localtime()
print(time.strftime("%Y-%m-%d %H:%M:%S",etime))
#####################

2019-01-22 10:36:00
2019-01-22 10:36:04


In [8]:
print( time.localtime( time.time() ) )

time.struct_time(tm_year=2019, tm_mon=1, tm_mday=22, tm_hour=10, tm_min=36, tm_sec=4, tm_wday=1, tm_yday=22, tm_isdst=0)


In [9]:
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) )

2019-01-22 10:36:04
